In [16]:
%load_ext autoreload
%autoreload 2

import os
import sys

module_path = os.path.abspath(os.path.join('../../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from generator import RoadNetwork, Trajectory
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import pandas as pd
import networkx as nx
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch_geometric.transforms as T

from models import TemporalGraphTrainer, GTCModel, Traj2VecModel

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
# 1. Load required data
unmapped_traj = pd.read_csv("../../../datasets/trajectories/hanover/temporal/mapped_id_poly_clipped.csv", ";")
traj = Trajectory("../../../datasets/trajectories/hanover/temporal/road_segment_map_final.csv", nrows=100000000).generate_TTE_datatset()
traj["seg_seq"] = traj["seg_seq"].map(np.array)
traj = traj.join(unmapped_traj[["start_stamp", "end_stamp", "id"]].set_index("id"), on="id", how="left")
network = RoadNetwork()
network.load_hanover_temporal(path="../../../datasets/trajectories/hanover/temporal/hannover_streetgraph.csv")

Pandas Apply:   0%|          | 0/61717 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/61717 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/61717 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/61717 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/61717 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/8620 [00:00<?, ?it/s]

In [24]:
network = network.subsample_graph(ratio=0.4)

In [ ]:
# Folium Heat Map
import folium
from folium import plugins
from folium.plugins import HeatMap
from collections import defaultdict
import branca.colormap

m = folium.Map(location=[52.37052, 9.73322],
                    zoom_start = 8)

sub_df["coords"] = sub_df["geometry"].swifter.apply(lambda x: list(x.coords))
coords = sub_df.loc[:, "coords"].values

for line in coords:
    data = [(c[1], c[0]) for c in line]
    folium.PolyLine(data, color="red", weight=2.5, opacity=0.8).add_to(m)

# coords = gdf["coords"].values
# for line in coords:
#     data = [(c[1], c[0]) for c in line]
#     folium.PolyLine(data, color="green", weight=2.5, opacity=0.8).add_to(m)

# Display the map
#map_porto.save("heatmap_gps_points_porto.html")
m

In [27]:
# extract relevant time data from time stamp
traj["start_stamp"] = pd.to_datetime(traj["start_stamp"], unit="s")
traj["end_stamp"] = pd.to_datetime(traj["end_stamp"], unit="s")

traj["dayofweek"] = traj["start_stamp"].dt.dayofweek
traj["start_hour"] = traj["start_stamp"].dt.hour
traj["end_hour"] = traj["end_stamp"].dt.hour

traj

,id,seg_seq,travel_time,start_stamp,end_stamp,dayofweek,start_hour,end_hour
0,1,"[8539, 7659, 6474, 8423, 3849, 7554, 975, 976,...",120,2019-11-25 22:18:49,2019-11-26 01:00:46,0,22,1
1,66668,"[7808, 7809, 4257, 4721, 7376, 7390, 8100, 797...",285,2019-11-03 16:18:34,2019-11-03 17:28:51,6,16,17
2,66669,"[1205, 896, 894, 3833, 4613, 416, 1285, 2818, ...",285,2019-11-03 16:18:34,2019-11-03 17:28:51,6,16,17
3,66670,"[1537, 1538, 4709, 5167, 6960, 7749]",142,2019-11-03 16:18:34,2019-11-03 17:28:51,6,16,17
4,3,"[3444, 3445, 1516, 1517, 3470, 3517, 3522, 296...",724,2019-11-15 11:45:02,2019-11-15 13:33:42,4,11,13
...,...,...,...,...,...,...,...,...
61712,118937,"[6183, 3053, 3055, 3479, 4806, 1045, 1047, 366...",197,2019-08-28 06:40:31,2019-08-28 07:21:36,2,6,7
61713,118943,"[89, 90, 7172, 8178, 6721, 3462, 3463, 8047, 5...",75,2019-08-17 08:38:15,2019-08-17 09:22:24,5,8,9
61714,118945,"[2139, 5747, 2836, 2837, 3747, 3748, 7584, 445...",84,2019-08-17 08:38:15,2019-08-17 09:22:24,5,8,9
61715,118950,"[8103, 245, 248, 788, 785, 459, 462, 6515, 518...",479,2019-08-19 15:15:23,2019-08-19 15:57:59,0,15,15


In [28]:
# precalc adj matrices
adj = GTCModel.generate_node_traj_adj(network=network, traj_data=traj, k=2, bidirectional=True, add_self_loops=True)
np.savetxt("./traj_adj_k_2_bi_temporal_gtc.gz", X=adj)

  0%|          | 0/61717 [00:00<?, ?it/s]


KeyError: 8539

In [4]:
# Pretrain tsd model 
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
adj_t2v = np.loadtxt("./traj_adj_k_1_for_temporal_tsd.gz")
print(adj_t2v.shape)
data = network.generate_road_segment_pyg_dataset(only_edge_index=True)
traj2vec = Traj2VecModel(
            data,
            network,
            adj=adj_t2v,
            device=device,
            emb_dim=128,
            walk_length=30,
            context_size=5,
            walks_per_node=25,
            num_neg=10,
        )
traj2vec.train(epochs=30)

(8620, 8620)
Epoch: 1, avg_loss: 8.289206834400401
Epoch: 2, avg_loss: 5.029554535360897
Epoch: 3, avg_loss: 3.7701260535156025
Epoch: 4, avg_loss: 3.081674549947767
Epoch: 5, avg_loss: 2.6459378025111033
Epoch: 6, avg_loss: 2.3453161246051977
Epoch: 7, avg_loss: 2.12550759115139
Epoch: 8, avg_loss: 1.9578257440863287
Epoch: 9, avg_loss: 1.8257293034028383
Epoch: 10, avg_loss: 1.718999320268631
Epoch: 11, avg_loss: 1.6309666211273581
Epoch: 12, avg_loss: 1.5571258721836643
Epoch: 13, avg_loss: 1.4943010152330227
Epoch: 14, avg_loss: 1.4402011797583405
Epoch: 15, avg_loss: 1.3931278799678768
Epoch: 16, avg_loss: 1.3517971623908074
Epoch: 17, avg_loss: 1.3152203686200215
Epoch: 18, avg_loss: 1.2826225056367764
Epoch: 19, avg_loss: 1.2533872964396935
Epoch: 20, avg_loss: 1.227023006975651
Epoch: 21, avg_loss: 1.203123044608688
Epoch: 22, avg_loss: 1.1813626874258176
Epoch: 23, avg_loss: 1.1614641379517365
Epoch: 24, avg_loss: 1.1431990923250426
Epoch: 25, avg_loss: 1.1263754868507387
Epoc

In [5]:
traj2vec.save_model(path="")

In [4]:
# load gtc adj
adj = np.loadtxt("./traj_adj_k_2_bi_temporal_gtc.gz")

In [30]:
print(adj.shape)

(8620, 8620)


In [56]:
import gc 
del model
gc.collect()
torch.cuda.empty_cache()

In [4]:
# load tsd pre emb
data = network.generate_road_segment_pyg_dataset(only_edge_index=True)
tsd = Traj2VecModel(data, network, device=device)
tsd.load_model("model_tsd.pt")
tsd_emb = tsd.load_emb()

NameError: name 'device' is not defined

In [5]:
torch.cuda.empty_cache()
# load train data
data = torch.load("../../../datasets/trajectories/hanover/temporal/temporal_data.pt")
data = torch.swapaxes(data, 0, 1)
data = data.numpy()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
edge_index = network.generate_road_segment_pyg_dataset(only_edge_index=True).edge_index
# initialize model
model = TemporalGraphTrainer(data=data, adj=adj, edge_index=edge_index, struc_emb=None, device=device, batch_size=12)

(5952, 8620, 4)


In [6]:
model.load_model("model.pt")

In [7]:
from prettytable import PrettyTable

def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: continue
        params = parameter.numel()
        table.add_row([name, params])
        total_params+=params
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params

count_parameters(model.model)

+---------------------------------------------+------------+
|                   Modules                   | Parameters |
+---------------------------------------------+------------+
| module.encoder._encoder.graph_conv1.weights |   33792    |
|  module.encoder._encoder.graph_conv1.biases |    256     |
| module.encoder._encoder.graph_conv2.weights |   16896    |
|  module.encoder._encoder.graph_conv2.biases |    128     |
|    module.decoder._tdecoder.weight_ih_l0    |   49152    |
|    module.decoder._tdecoder.weight_hh_l0    |   49152    |
|     module.decoder._tdecoder.bias_ih_l0     |    384     |
|     module.decoder._tdecoder.bias_hh_l0     |    384     |
|        module.decoder.dense.0.weight        |    8192    |
|         module.decoder.dense.0.bias         |     64     |
|        module.decoder.dense.2.weight        |     64     |
|         module.decoder.dense.2.bias         |     1      |
+---------------------------------------------+------------+
Total Trainable Params: 

158465

In [ ]:
# train model
model.train(epochs=10)

In [24]:
torch.save(model.model.state_dict(), os.path.join("model.pt"))

In [8]:
data.shape

(5952, 8620, 4)

In [43]:
# test sequence encoding 
# z, _ = model.model(torch.Tensor(data[100:112]).unsqueeze(0))
z = z.squeeze()
z.shape

torch.Size([8620, 128])

In [44]:
from sklearn import model_selection
from sklearn import linear_model
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer

# n2v = models[-1]
idxs = np.arange(len(network.line_graph.nodes))
train_idx, test_idx = model_selection.train_test_split(idxs, test_size=0.2, random_state=69)
y = np.array([network.gdf_edges.loc[n]["highway_enc"] for n in network.line_graph.nodes])

# for m, e in models:
    # m.train(epochs=e)
    # zn = m.load_emb()
    # zcn = np.concatenate((zn, z2), axis=1)
    # zct = np.concatenate((zn, z3), axis=1)
    # zcnn = np.concatenate((zn, z4), axis=1)
    # zctn = np.concatenate((zn, z5), axis=1)
    # X = z # embedding for each node
eva = [z.detach().cpu().numpy()] # gtc.load_emb(), gae_emb, rand_emb
for X in eva:
    X_train, X_test, y_train, y_test = X[train_idx], X[test_idx], y[train_idx], y[test_idx]

    lm = linear_model.LogisticRegression(multi_class="multinomial", max_iter=1000)
    scorer = make_scorer(metrics.f1_score, average="macro")
    print(np.mean(cross_val_score(estimator=lm, X=X, y=y, scoring=scorer, cv=5)))

0.5542513908639789


# Test on traveltime task

In [8]:
from torch.utils.data import DataLoader, Dataset

from sklearn import model_selection


class TemporalDataset(Dataset):
    def __init__(self, data, network):
        self.X = data["seg_seq"].values
        self.y = data["travel_time"].values
        self.time = data[["dayofweek", "start_hour", "end_hour"]].values
        self.network = network
        self.map = self.create_edge_emb_mapping()

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        return torch.tensor(self.X[idx], dtype=int), self.y[idx], self.time[idx], self.map

    # tested index mapping is correct
    def create_edge_emb_mapping(self):
        # map from trajectory edge id to embedding id
        # edge_ids = np.array(self.network.gdf_edges.index, dtype="i,i,i")
        # traj_edge_idx = np.array(self.network.gdf_edges.fid)
        # node_ids = np.array(self.network.line_graph.nodes, dtype="i,i,i")
        # sort_idx = node_ids.argsort()
        # emb_ids = sort_idx[np.searchsorted(node_ids, edge_ids, sorter=sort_idx)]
        # map = dict(zip(traj_edge_idx, emb_ids))

        map = {}
        nodes = list(self.network.line_graph.nodes)
        for index, id in zip(self.network.gdf_edges.index, self.network.gdf_edges.fid):
            map[id] = nodes.index(index)
        # print(map == map2) # yields true

        return map

    @staticmethod
    def collate_fn_padd(batch):
        """
        Padds batch of variable length
        """
        data, label, time,  map = zip(*batch)
        # seq length for each input in batch
        lengths_old = torch.tensor([t.shape[0] for t in data])

        # sort data for pad packet, since biggest sequence should be first and then descending order
        sort_idxs = torch.argsort(lengths_old, descending=True, dim=0)
        lengths = lengths_old[sort_idxs]
        data = [
            x
            for _, x in sorted(
                zip(lengths_old.tolist(), data), key=lambda pair: pair[0], reverse=True
            )
        ]
        label = [
            x
            for _, x in sorted(
                zip(lengths_old.tolist(), label), key=lambda pair: pair[0], reverse=True
            )
        ]
        time = [
            x
            for _, x in sorted(
                zip(lengths_old.tolist(), time), key=lambda pair: pair[0], reverse=True
            )
        ]

        # pad
        data = torch.nn.utils.rnn.pad_sequence(data, padding_value=0, batch_first=True)
        # compute mask
        mask = data != 0

        return data, torch.Tensor(label), torch.tensor(time, dtype=int), lengths, mask, map[0]


train, test = model_selection.train_test_split(
            traj, test_size=0.2, random_state=69
        )
train_loader = DataLoader(
    TemporalDataset(train, network),
    collate_fn=TemporalDataset.collate_fn_padd,
    batch_size=128,
    shuffle=True,
)
eval_loader = DataLoader(
    TemporalDataset(test, network),
    collate_fn=TemporalDataset.collate_fn_padd,
    batch_size=128,
)

In [9]:
from operator import itemgetter
from torch.autograd import Variable

# Init 
class TemporalTest(nn.Module):
    def __init__(
        self,
        device,
        emb_dim: int = 128,
        hidden_units: int = 128,
        layers: int = 2,
        batch_size: int = 128,
        plugin=None,
    ):
        super(TemporalTest, self).__init__()
        self.encoder = nn.LSTM(
            emb_dim, hidden_units, num_layers=layers, batch_first=True, dropout=0.5
        )
        if plugin is not None:
            hidden_units = hidden_units * 2
        self.decoder = nn.Sequential(
            nn.Linear(hidden_units, hidden_units * 2),
            nn.ReLU(),
            nn.Linear(hidden_units * 2, hidden_units),
            nn.ReLU(),
            nn.Linear(hidden_units, 1),
        )
        self.hidden_units = hidden_units
        self.layers = layers
        self.batch_size = batch_size
        self.device = device
        self.loss = nn.MSELoss()
        self.opt = torch.optim.Adam(self.parameters(), lr=0.001)
        self.plugin = plugin

        self.encoder.to(device)
        self.decoder.to(device)

    def forward(self, x, lengths):
        batch_size, seq_len, _ = x.size()
        self.hidden = self.init_hidden(batch_size=batch_size)

        x = torch.nn.utils.rnn.pack_padded_sequence(x, lengths, batch_first=True)

        x, _ = self.encoder(x)

        x, plengths = torch.nn.utils.rnn.pad_packed_sequence(
            x, batch_first=True, padding_value=0
        )
        x = x.contiguous()  # batch x seq x hidden
        # x = x.view(-1, x.shape[2])
        x = torch.stack(
            [x[b, plengths[b] - 1] for b in range(batch_size)]
        )  # get last valid item per batch batch x hidden

        if self.plugin is not None:
            x = self.plugin(x)

        yh = self.decoder(x)

        return yh  # (batch x 1)

    def train_model(self, loader, epochs=100):
        self.train()
        for e in range(epochs):
            total_loss = 0
            for X, y, time, lengths, mask, map in loader:

                emb = self.plugin.generate_emb(time)
    
                emb_batch = self.get_embedding(emb, X.clone(), mask, map)
                emb_batch = emb_batch.to(self.device)

                y = y.to(self.device)
                yh = self.forward(emb_batch, lengths)
                loss = self.loss(yh.squeeze(), y)

                self.opt.zero_grad()
                loss.backward()
                self.opt.step()
                total_loss += loss.item()

                break
            break

            # print(f"Average training loss in episode {e}: {total_loss/len(loader)}")

    def predict(self, loader, emb):
        with torch.no_grad():
            self.eval()
            yhs, ys = [], []
            for X, y, lengths, mask, map in loader:
                emb_batch = self.get_embedding(emb, X.clone(), mask, map)
                emb_batch = emb_batch.to(self.device)

                if self.plugin is not None:
                    self.plugin.register_id_seq(X, mask, map, lengths)

                y = y.to(self.device)
                yh = self.forward(emb_batch, lengths)
                yhs.extend(yh.tolist())
                ys.extend(y.tolist())

            return np.array(yhs), np.array(ys)

    def init_hidden(self, batch_size):
        hidden_a = torch.randn(self.layers, batch_size, self.hidden_units)
        hidden_b = torch.randn(self.layers, batch_size, self.hidden_units)

        hidden_a = Variable(hidden_a)
        hidden_b = Variable(hidden_b)

        hidden_a = hidden_a.to(self.device)
        hidden_b = hidden_b.to(self.device)

        return (hidden_a, hidden_b)

    def get_embedding(self, emb, batch, mask, map):
        """
        Transform batch_size, seq_length, 1 to batch_size, seq_length, emb_size
        """
        res = torch.zeros((batch.shape[0], batch.shape[1], emb.shape[1]))
        for i, seq in enumerate(batch):
            emb_ids = itemgetter(*seq[mask[i]].tolist())(map)
            res[i, mask[i], :] = torch.Tensor(emb[emb_ids, :]).float()

        return res


In [10]:
# load temporal sensor data 
temporal = pd.read_csv("../../../datasets/trajectories/hanover/temporal/hannover_traffic.csv")

In [12]:
# initialize temporal plugin 
from evaluation.tasks import TemporalEmbeddingPlugin

plugin = TemporalEmbeddingPlugin(model.model, network, device)

In [13]:
plugin.load_data("plugin_data.pt")

In [14]:
plugin.processed_temp_data.shape

torch.Size([672, 8620, 4])

In [12]:
# save preprocessed 
torch.save(plugin.processed_temp_data, 'plugin_data.pt')

In [15]:
test_traj = traj.iloc[0, [-1, -2, -3]].values

z, _ = plugin.generate_emb(test_traj[np.newaxis, ...])

In [16]:
task = TemporalTest(
            plugin=plugin,
            device=device,
            batch_size=4,
        )

In [17]:
task.train_model(loader=train_loader, epochs=10)

RuntimeError: Caught RuntimeError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/home/pheinemeyer/miniconda3/envs/road/lib/python3.9/site-packages/torch/nn/parallel/parallel_apply.py", line 61, in _worker
    output = module(*input, **kwargs)
  File "/home/pheinemeyer/miniconda3/envs/road/lib/python3.9/site-packages/torch/nn/modules/module.py", line 1110, in _call_impl
    return forward_call(*input, **kwargs)
  File "/dstore/home/pheinemeyer/Road-Network-Embedding-Generator/models/tgtc.py", line 133, in forward
    recon = self.decoder(z, X.shape[1])
  File "/home/pheinemeyer/miniconda3/envs/road/lib/python3.9/site-packages/torch/nn/modules/module.py", line 1110, in _call_impl
    return forward_call(*input, **kwargs)
  File "/dstore/home/pheinemeyer/Road-Network-Embedding-Generator/models/tgtc.py", line 155, in forward
    x, _ = self._tdecoder(z)
  File "/home/pheinemeyer/miniconda3/envs/road/lib/python3.9/site-packages/torch/nn/modules/module.py", li